In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
df = pd.read_excel('D:/documents/1_projects/kombarov/input/input.xlsx')

df = df.rename(columns={
    # 'Накопленная добыча жидкости':'накопленная_добыча_жидкости',
    # 'Накопленная добыча нефти':'накопленная_добыча_нефти',
    # 'Накопленная добыча воды':'накопленная_добыча_воды',
    # 'Годовая добыча жидкости':'годовая_добыча_жидкости',
    # 'Годовая добыча нефти':'годовая_добыча_нефти',
    # 'Годовая добыча воды':'годовая_добыча_воды'
    'Накопленная добыча жидкости':'q_ж',
    'Накопленная добыча нефти':'q_н',
    'Накопленная добыча воды':'q_в',
    'Годовая добыча жидкости':'d_q_ж',
    'Годовая добыча нефти':'d_q_н',
    'Годовая добыча воды':'d_q_в'
})

Перевод единиц в м3

In [73]:
# исходные параметры
# ro_n_pov = 0.84
# ro_v_pov = 1.19
# q_geol = 8958.4
# q_izvl = 5491.47
# ob_koef = 1.202
ro_n_pov = 0.93
ro_v_pov = 1.15
q_geol = 12833
q_izvl = 6701
ob_koef = 1.04

# перевод тыс тонн в метры кубические, умножаем на 1000 и делим на плотность
for line in df.index:
    df.loc[line, 'q_н'] = round(df.loc[line, 'q_н'] * 1000 / ro_n_pov * ob_koef, 3)
    df.loc[line, 'd_q_н'] = round(df.loc[line, 'd_q_н'] * 1000 / ro_n_pov * ob_koef, 3)
    
    df.loc[line, 'q_в'] = round(df.loc[line, 'q_в'] * 1000 / ro_v_pov, 3)
    df.loc[line, 'd_q_в'] = round(df.loc[line, 'd_q_в'] * 1000 / ro_v_pov, 3)
    
    df.loc[line, 'q_ж'] = round(df.loc[line, 'q_в'] + df.loc[line, 'q_н'], 3)
    df.loc[line, 'd_q_ж'] = round(df.loc[line, 'd_q_в'] + df.loc[line, 'd_q_н'], 3)
    
    

# для геологических и извлекаемых запасов
q_geol = round(q_geol * 1000 / ro_n_pov * ob_koef, 3)
q_izvl = round(q_izvl * 1000 / ro_n_pov * ob_koef, 3)


In [74]:
df.head()

,год,q_ж,q_н,q_в,d_q_ж,d_q_н,d_q_в
0,2019,380561.196,113169.892,267391.304,104918.186,47526.882,57391.304
1,2020,529358.018,120662.366,408695.652,110135.764,48309.677,61826.087
2,2021,749332.772,170984.946,578347.826,228505.657,88679.570,139826.087


1. Начальные извлекаемые запасы нефти численно равны коэффициенту a:

In [75]:
a = ((df.loc[2, 'q_н'] * df.loc[2, 'q_ж']) + (df.loc[1, 'q_н'] * df.loc[1, 'q_ж']) - 2 * (df.loc[0, 'q_н'] * df.loc[0, 'q_ж']))/\
    (df.loc[2, 'q_ж'] + df.loc[1, 'q_ж'] - 2 * df.loc[0, 'q_ж'])
b = a * df.loc[2, 'q_ж'] - (df.loc[2, 'q_н'] * df.loc[2, 'q_ж'])
print('a= ', a)
print('b= ', b)

a=  204537.36351959265
b=  25141926027.257736


In [98]:
df['B'] = 0
df['B_ves'] = 0

for line in range(3, 10):
    # 2.Определяем годовую добычу нефти
    df.loc[line, 'd_q_н'] = a - (b / (df.loc[2, 'q_ж'] + df.loc[2, 'd_q_ж'] * (line - 2))) - df.loc[line - 1, 'q_н']
    
    # Определяем добычу попутной воды по годам прогнозного периода
    df.loc[line, 'd_q_в'] = df.loc[2, 'd_q_ж'] - df.loc[line, 'd_q_н']
    
    # 4. Определяем среднегодовую объемную обводнённость добываемой жидкости, %
    df.loc[line, 'B'] = df.loc[line, 'd_q_в'] / df.loc[2, 'd_q_ж'] * 100
    
    # 5. Рассчитываются накопленные отборы нефти, воды и жидкости в пластовых
# условиях по годам прогнозируемого периода
    df.loc[line, 'q_н'] = df.loc[line - 1, 'q_н'] + df.loc[line, 'd_q_н']
    df.loc[line, 'q_в'] = df.loc[line - 1, 'q_в'] + df.loc[line, 'd_q_в']
    
    df.loc[line, 'd_q_ж'] = df.loc[line - 1, 'd_q_ж'] + df.loc[2, 'd_q_ж']
    df.loc[line, 'q_ж'] = df.loc[line - 1, 'q_ж'] + df.loc[line, 'd_q_ж']
    
    # год 
    df.loc[line, 'год'] = df.loc[line - 1, 'год'] + 1
    
    # 6. Рассчитывается весовая обводненность аналогично пункту 4,
    df.loc[line, 'B_ves'] = (df.loc[line, 'd_q_в'] * ro_v_pov) / (df.loc[2, 'd_q_в'] * ro_v_pov + df.loc[2, 'd_q_н'] * ro_n_pov) * 100

In [103]:
df

,год,q_ж,q_н,q_в,d_q_ж,d_q_н,d_q_в,B,B_ves
0,2019.0,380561.196,113169.892000,2.673913e+05,104918.186,47526.882000,57391.304000,0.000000,0.000000
1,2020.0,529358.018,120662.366000,4.086957e+05,110135.764,48309.677000,61826.087000,0.000000,0.000000
2,2021.0,749332.772,170984.946000,5.783478e+05,228505.657,88679.570000,139826.087000,0.000000,0.000000
3,2022.0,1206344.086,178825.624973,7.990128e+05,457011.314,7840.678973,220664.978027,96.568716,104.313166
4,2023.0,1891861.057,183695.941641,1.022648e+06,685516.971,4870.316668,223635.340332,97.868623,105.717321
5,2024.0,2805883.685,187015.022836,1.247835e+06,914022.628,3319.081195,225186.575805,98.547484,106.450624
6,2025.0,3948411.970,189422.177657,1.473933e+06,1142528.285,2407.154821,226098.502179,98.946567,106.881712
7,2026.0,5319445.912,191247.843164,1.700613e+06,1371033.942,1825.665507,226679.991493,99.201041,107.156594
8,2027.0,6918985.511,192680.015514,1.927687e+06,1599539.599,1432.172350,227073.484650,99.373244,107.342607
9,2028.0,8747030.767,193833.535445,2.155039e+06,1828045.256,1153.519931,227352.137069,99.495190,107.474332


In [104]:
df.to_excel('D:/documents/1_projects/kombarov/result/result_komb.xlsx', sheet_name='main')